In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from collections import deque 
import os

## Load and clean the data

In [5]:
lang_map = {'de' : 0, 'en': 1, 'es': 2, 'fr': 3, 'it': 4, 'pt': 5}

if not os.path.exists("data/cleaned.csv"):
    df = pd.read_csv("data/settles.acl16.learning_traces.13m.csv")

    df.sort_values(by=['user_id', 'lexeme_id', 'timestamp'], inplace=True)
    
    #Drop this column as it's inferred from last two
    df = df.drop(["p_recall"], axis=1)

    #Hash lexemes for smaller storage
    df['lexeme_id'] = df['lexeme_id'].apply(hash) % 1000000
    
    #Hash user id's for smaller storage
    df['user_id'] = df['user_id'].apply(hash) % 5000000
    
    #Map languages to numbers for smaller storage
    df['learning_language'] = df['learning_language'].map(lang_map)
    df['ui_language'] = df['ui_language'].map(lang_map)
    
    for c in df.columns:
        if c != 'lexeme_string':
            df[c] = pd.to_numeric(df[c], downcast='unsigned')
    
    
    
    
    df['lexeme_string'] = df.lexeme_string.map(lambda x: x[0: x.find('<')])
    df.to_csv("data/cleaned.csv", index=False)
else:
    df = pd.read_csv("data/cleaned.csv")
    for c in df.columns:
        if c != 'lexeme_string':
            df[c] = pd.to_numeric(df[c], downcast='unsigned')
    

### If data already cleaned run me instead

## Scheduling Simulator

In [4]:
class Scheduler:
    """
    Parent class of any learning scheduler method.
    """
    
    def __init__(self, num_items):
        pass
    
    def next_item(self):
        pass
    
    def update(self, item, outcome):
        pass
    

class Random(Scheduler):
    """
    Scheduler that selects random items to present.
    """
    def __init(self, num_items):
        self.n = num_items
    
    def next_item(self):
        return np.random.randint(0, num_items)
    
    def update(self, item, outcome):
        pass
        
        

class Leitner(Scheduler): 
    """
    This class implements a Leitner scheduler that samples from 
    boxes with exponentially decreasing probability. Cards enter
    in box 0 and leave when they are correctly answered after entering 
    the final box
    """
    def __init__(self, nb):
        '''
        :param nb: Number of boxes
        boxes is a list of queues representing the boxes.
        dist_boxes is sampling distribution for which box to select fromr
        cards is a set of items in the boxes currently.
        '''
        self.boxes = [deque() for _ in nb]
        self.dist_boxes = np.array([1/2**i for i in range(nb)]) / sum([1/2**i for i in range(nb)])
        self.cards = set()
        
    
    def next_item(self):
        """
        Gets the next item in the learning sequence.
        """
        self.recent_box = np.random.multinomial(1, self.dist_boxes).argmax()
        
        if len(self.boxes[self.recent_box]):
            return self.boxes[self.recent_box].pop()
        else:
            return self.next_item()
    
    def update(self, item, outcome, thresh=.9):
        """
        Updates the most recent item from the sequence
        by putting it back depending on the outcome.
        """
        if outcome > thresh:
            new_box = self.recent_box + 1
            if new_box >= len(self.boxes):
                self.cards.remove(item)
            else:
                self.boxes[new_box].appendleft(item)
        else:
            new_box = max(self.recent_box - 1, 0)
            
            self.boxes[new_box].appendleft(item)
        
            


## Sample trajectories from historical data

In [7]:
df.head(50)

,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct
0,1362206313,8337322,1787641,0,1,791999,katze/katze,3,3,1,0
1,1362206313,16777591,1787641,0,1,158877,dem/das,2,1,1,1
2,1362206313,19628054,1787641,0,1,911450,frau/frau,15,14,1,1
3,1362206313,8346489,1787641,0,1,591433,esse/essen,11,11,1,1
4,1362206313,6842117,1787641,0,1,652769,alles/alle,2,2,1,1
5,1362206313,21350328,1787641,0,1,663503,mag/mögen,1,1,1,1
6,1362206313,8346489,1787641,0,1,410916,isst/essen,10,10,1,1
7,1362206313,6530800,1787641,0,1,815892,mutter/mutter,3,3,1,1
8,1362206313,21705984,1787641,0,1,81521,spiele/spielen,1,1,4,3
9,1362206313,20674367,1787641,0,1,617438,mädchen/mädchen,7,6,1,1


In [31]:
df.sort_values(by=["user_id", "lexeme_id", "timestamp"], inplace=True)

In [11]:
df.iloc[50:100, :]

,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct
50,1362491828,482,2159030,3,1,278850,écrivons/écrire,3,3,2,2
51,1362490845,495500,2159030,3,1,777312,vous/vous,12,10,5,5
52,1362491828,482,2159030,3,1,377933,lisent/lire,3,3,4,3
53,1362653855,14004445,20728,2,1,648756,además/además,5,5,2,2
54,1362653855,13354816,20728,2,1,866034,bastante/bastante,6,4,2,2
55,1362652969,13354316,20728,2,1,977091,van/ir,10,8,1,1
56,1362652486,9416405,20728,2,1,108533,tu/tuyo,41,36,2,1
57,1362652969,484,20728,2,1,108533,tu/tuyo,43,37,2,2
58,1362653855,886,20728,2,1,108533,tu/tuyo,45,39,1,0
59,1362653855,12679857,20728,2,1,865578,grande/grande,27,26,1,1


In [12]:
def timestamp_to_session(x):
    result = pd.DataFrame()
    
    timestamps_sorted = np.array(sorted(list(set(x['timestamp']))))    
    
    result['timestamp'] = timestamps_sorted
    result['session'] = list(range(len(timestamps_sorted)))
    return result


In [14]:
timestamp_map = df.loc[:, ['user_id', 'timestamp']].groupby(['user_id']).apply(timestamp_to_session)

In [ ]:
timestamp_map.head(50)

In [ ]:
lexeme_difficulties = df[['session_correct', 'session_seen']].groupby('lexeme_id').agg(sum)
lexeme_difficulties['score'] = lexeme_difficulties['session_correct']/lexeme_difficulties['session_seen']

In [2]:
class SampleEnv():
    
    def __init__(self, df):
        self.df = df
        # State: row corresponds to lexeme id, first column is history_seen, second column is history_correct
        self.state = np.zeros((1000000, 2))
        self.users = df['user_id'].unique()
        self.num_users = len(users)
        self.num_lexemes = lexeme_difficulties.shape[0]
        
    def sample_trajectory(self):
        user = self.users[np.random.choice(len(self.users))]
        points = self.df[self.df['user_id'] == user]
        
        obs, actions = [], []
        idx = 0
        point = points.iloc[idx]
        while point['user_id'] == user and idx < points.shape[0]-1:
            lex = points['lexeme_id']
            self.state[lex][0] = point['history_seen']
            self.state[lex][1] = point['history_correct']
            self.state[lex][2] = lexeme_difficulties[lex]
            obs.append(self.state.copy())
            actions = np.zeros((self.num_lexemes))
            actions.append(points.iloc[idx+1]['lexeme_id'])
            idx += 1
            point = points.iloc[idx]
        return obs, actions

In [9]:
env = SampleEnv(df)

In [10]:
print(env.sample_trajectory())

([array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]])], [328895, 826997, 737582, 235704, 455383, 399055])
